In [36]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [38]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [39]:
from core.agent import *
from core.world import * 
from dynamics.space.resource_map import *
from core.render import * 
from core.env_params import * 
from core.skill import * 
from core.models import *
from core.env_params import *
from core.message import *

In [40]:
from dynamics.agents.sar_agent import *
from dynamics.agents.trade_agent import * 
from dynamics.world.sar_world import *
from dynamics.models.util_sampler import *

def initialize_swarm(world : BaseWorld):
    swarm = [SARAgent() for i in range(SWARM_SIZE)]
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)


In [41]:

from urban.urban_gen import * 
from dynamics.models.victims import * 
from urban.sar_comm import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)
def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [42]:
from dynamics.models.energy import EnergyModel
from dynamics.models.victims import VictimModel
from solution.sar_action_interpreter import *

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

comms_protocol = SARCommunicationProtocol()
action_interpreter = SARActionInterpreter()

In [43]:
custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter)

In [44]:
custom_gym.reset(42)

({1: {'victims': array([[0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 1., 1., 0., 1.],
          [0., 0., 1., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 1., 1., 0.],
          [0., 1., 1., 1., 0., 0., 0.],
          [0., 0., 1., 1., 1., 1., 0.]])},
  2: {'victims': array([[1., 1., 0., 1., 1., 1., 1.],
          [0., 1., 1., 0., 1., 1., 1.],
          [0., 0., 0., 1., 0., 1., 0.],
          [1., 1., 1., 0., 1., 1., 1.],
          [1., 1., 0., 0., 0., 1., 0.],
          [0., 1., 0., 1., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0., 1.]])},
  3: {'victims': array([[0., 0., 0., 0., 1., 1., 1.],
          [0., 1., 0., 1., 1., 1., 1.],
          [1., 0., 0., 1., 1., 1., 0.],
          [1., 0., 0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0., 1., 1.],
          [0., 1., 1., 0., 0., 1., 0.],
          [0., 1., 0., 1., 1., 0., 1.]])},
  4: {'victims': array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 1., 0.],
          [0., 

### Testing

In [45]:
parallel_api_test(custom_gym, num_cycles=1_000_000)
custom_gym.reset()

Passed Parallel API test


({1: {'victims': array([[0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 1., 1., 0., 1.],
          [0., 0., 1., 0., 0., 1., 0.],
          [0., 1., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 1., 1., 0.],
          [0., 1., 1., 1., 0., 0., 0.],
          [0., 0., 1., 1., 1., 1., 0.]])},
  2: {'victims': array([[1., 1., 0., 1., 1., 1., 1.],
          [0., 1., 1., 0., 1., 1., 1.],
          [0., 0., 0., 1., 0., 1., 0.],
          [1., 1., 1., 0., 1., 1., 1.],
          [1., 1., 0., 0., 0., 1., 0.],
          [0., 1., 0., 1., 0., 0., 1.],
          [0., 0., 0., 0., 0., 0., 1.]])},
  3: {'victims': array([[0., 0., 0., 0., 1., 1., 1.],
          [0., 1., 0., 1., 1., 1., 1.],
          [1., 0., 0., 1., 1., 1., 0.],
          [1., 0., 0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0., 1., 1.],
          [0., 1., 1., 0., 0., 1., 0.],
          [0., 1., 0., 1., 1., 0., 1.]])},
  4: {'victims': array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 1., 0.],
          [0., 

# Training

In [46]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet

In [47]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             policy_net= PolicyNet(1, 7, 12), 
             target_net=  PolicyNet(1, 7, 12))

In [48]:
train_loop(custom_gym, model, games=10, seed=42)

Training on thesis.


KeyError: 'Victims'